In [1]:
import os
import os.path
import random
import numpy as np
import PIL
import cv2
import cv2.xfeatures2d
import matplotlib.pyplot as plt
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
def image_path_itr(keyword):
    dir_ = 'downloads/{}/'.format(keyword)
    for f in os.listdir(dir_):
        if not f.endswith('.jpg'):
            continue
        yield dir_ + f

In [3]:
def pil_image_to_numpy(im):
    im = np.array(im.convert('RGB'), dtype=np.uint8)[:,:,::-1]
    return im

In [4]:
def extract_features(keyword):
    surf = cv2.xfeatures2d.SURF_create(10000)
    features = []
    for p in image_path_itr(keyword):
        try:
            im = pil_image_to_numpy(PIL.Image.open(p))
            f = surf.detectAndCompute(im, None)
            if not f[0]:
                continue
            features.append(f)
        except:
            pass
    return features

In [5]:
samples = \
    [(f, 0) for f in extract_features('dragonfly')] + \
    [(f, 1) for f in extract_features('butterfly')]
random.shuffle(samples)
len(samples)

191

In [6]:
train_samples = samples[:-40]
valid_samples = samples[-40:]
train_X = [s[0][1] for s in train_samples]
train_Y = [keras.utils.to_categorical(s[1], num_classes=2) for s in train_samples]
valid_X = [s[0][1] for s in valid_samples]
valid_Y = [keras.utils.to_categorical(s[1], num_classes=2) for s in valid_samples]

In [30]:
K.clear_session()
X = X_input = keras.layers.Input((None, 64))
X = keras.layers.BatchNormalization()(X)
#X = keras.layers.Dropout(0.5)(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(256, activation='relu'))(X)
X = X_desc = keras.layers.TimeDistributed(keras.layers.Dense(128, activation='relu'))(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='relu'))(X)
X = keras.layers.Softmax(axis=1)(X)
X = keras.layers.Dot(1)([X, X_desc])
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(2, activation='softmax')(X)
M = keras.models.Model([X_input], [X])
M.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 64)     0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, 64)     256         input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 256)    16640       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, None, 128)    32896       time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distr

In [31]:
for _ in range(100):
    trainning_set = list(zip(train_X, train_Y))
    random.shuffle(trainning_set)
    for X, Y in trainning_set:
        M.fit(
            np.expand_dims(X, axis=0),
            np.expand_dims(Y, axis=0),
            batch_size=1, verbose=False)
    n_correct = 0
    for X, Y in zip(valid_X, valid_Y):
        pred = M.predict(
            np.expand_dims(X, axis=0),
            batch_size=1)[0]
        if np.sum(pred * Y) > 0.5:
            n_correct += 1
    print('correct predicting ratio on valid. set', n_correct / len(valid_Y))

correct predicting ratio on valid. set 0.675
correct predicting ratio on valid. set 0.825
correct predicting ratio on valid. set 0.85
correct predicting ratio on valid. set 0.725
correct predicting ratio on valid. set 0.9
correct predicting ratio on valid. set 0.825
correct predicting ratio on valid. set 0.775
correct predicting ratio on valid. set 0.85
correct predicting ratio on valid. set 0.875
correct predicting ratio on valid. set 0.875
correct predicting ratio on valid. set 0.875
correct predicting ratio on valid. set 0.9
correct predicting ratio on valid. set 0.9
correct predicting ratio on valid. set 0.9
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting ratio on valid. set 0.925
correct predicting r